In [1]:
import json
import time
from bs4 import BeautifulSoup as bs
import requests

In [2]:
def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
        
def load_data(title):
    with open(title, encoding="utf-8") as f:
        return json.load(f)

# Movies
Using api to get the duration of each movie in the list below and save it to a Json then calculate total time wasted watching movies

In [3]:
all_movies_raw="""insert movies here separated by comma"""

In [4]:
all_movies = all_movies_raw.split(',')
all_movies = [x.strip().replace('\n',"") for x in all_movies]
# print(all_movies)

In [5]:
def get_movie_minutes_runtime(movie_name):
    api_key='personal-api-key-here'
    base_url=f'http://www.omdbapi.com/?apikey={api_key}&t='
    movie_url=movie_name.replace(' ','+')
    full_path=base_url+movie_url
    r = requests.get(full_path)
    content= json.loads(r.text)
    mins=content['Runtime'].split(' ')[0]
    return mins


In [6]:
movie_minutes={}
for index,movie_name in enumerate(all_movies):
    # if(index == 10):
    #     break
    try:
        minutes=get_movie_minutes_runtime(movie_name)
        movie_minutes[movie_name]= minutes
    except Exception as e:
        print(movie_name,e)

insert movies here separated by comma 'Runtime'


In [7]:
def get_invalid_value_dict(dict):
    for key, value in dict.items():
        if( (not str(value).isdigit()) or int(value)<40):
            print("Invalid",key)
            
get_invalid_value_dict(movie_minutes)

In [8]:
time_wasted_seen_minutes=0
for key,value in movie_minutes.items():
    try:
        time_wasted_seen_minutes+=int(value)
    except Exception as e:
        print(e)
time_wasted_seen_minutes

0

In [9]:
save_data("movie_data_watchtime_minutes.json", movie_minutes)
total_hours_wasted_seen=(time_wasted_seen_minutes)//60
print(total_hours_wasted_seen)

movies_list_txt=[movie.lower() for movie in movie_minutes]
movies_list_txt=sorted(movies_list_txt)
print(*movies_list_txt, sep=', ')

0



# TV shows

Using a web scraper on the website below to get the duration of each TV and transform it into hours

In [10]:
all_shows_raw="""tv shows here separated by comma"""
all_shows=all_shows_raw.split(',')
all_shows=[x.strip() for x in all_shows]
all_shows

['tv shows here separated by comma']

In [11]:
r=requests.get('https://www.bingeclock.com/s/game-of-thrones/')
# print(r.text)
soup = bs(r.content)
info_divs= soup.find_all("div", {"class": "date_counter_cont"})

def get_content_value_list(info_divs):
    list_value = []
    for row in info_divs:
        text=row.get_text()
        list_value.append(text)
    return list_value

info_divs_content = get_content_value_list(info_divs)
print(info_divs_content)

def time_element_to_minute(element):
    elements=element.strip().split(' ')
    # print(elements)
    if(elements[1]=='minutes'):
        return int(elements[0])
    elif(elements[1]=='hours' or elements[1]=='hour'):
        return int(elements[0])*60
    elif(elements[1]=='days' or elements[1]=='day'):
        return int(elements[0])*24*60
    else:
        raise ValueError("Invalid format for time element(doesnt match minutes,hour,days/day)")

print(time_element_to_minute(info_divs_content[0]))

def get_minutes_from_date_divs(info_divs):
    info_divs_content = get_content_value_list(info_divs)
    total_minutes=0
    for element in info_divs_content:
        total_minutes+=time_element_to_minute(element)
    
    return total_minutes

get_minutes_from_date_divs(info_divs)
   

[' 2 days ', ' 22 hours ', ' 14 minutes ']
2880


4214

In [12]:
def get_time_spent_minutes(url):
    r=requests.get(url)
    soup=bs(r.content)
    info_divs= soup.find_all("div", {"class": "date_counter_cont"})
    return get_minutes_from_date_divs(info_divs)
    
get_time_spent_minutes('https://www.bingeclock.com/s/game-of-thrones/')

4214

In [13]:
base_url = 'https://www.bingeclock.com/s/'
show_times={}
for index,show in enumerate(all_shows):
    # if index == 3:
    #     break
    show_url =show.replace(' ','-')
    full_url =base_url + show_url
    
    try:
        show_times[show] =get_time_spent_minutes(full_url)
    
    except Exception as e:
        print(show,e)
    time.sleep(1)
    # print(full_url)
    
show_times

{'tv shows here separated by comma': 0}

In [14]:
show_times_hours={}
for k,v in show_times.items():
    show_times_hours[k] =v//60
    
save_data("show_watchtime_hours.json",show_times_hours)
show_times_hours

{'tv shows here separated by comma': 0}

In [15]:
total_hours_wasted=0
for k,v in show_times_hours.items():
    total_hours_wasted+=v
print(total_hours_wasted)

tv_show_list=[x.strip().lower() for x in show_times_hours]
tv_show_list=sorted(tv_show_list)
print(*tv_show_list,sep=', ')

0
tv shows here separated by comma


# Anime

Usin Jikan's API to extract anime data

In [16]:
all_animes_raw=""" all animes here separated by commas"""

In [17]:
all_animes = all_animes_raw.split(',')
all_animes = [x.strip().replace('\n',"") for x in all_animes]
all_animes

['all animes here separated by commas']

In [18]:
def get_anime_episodes(anime_name):
    base_url="https://api.jikan.moe/v3/search/anime?q="
    full_path=base_url+anime_name
    r=requests.get(full_path)
    content=json.loads(r.text)
    first_anime=content['results'][0]
    if first_anime['type']=='Movie':
        return (first_anime['title'],4)#assume movie anime is 4 episodes long
    # return first_anime
    return (first_anime['title'],first_anime['episodes'])

get_anime_episodes("Owari no Seraph")

('Owari no Seraph', 12)

In [19]:
anime_episodes={}
for index,anime_name in enumerate(all_animes):
    # if(index == 10):
    #     break
    try:
        name,episodes=get_anime_episodes(anime_name)
        if name in anime_episodes:
            raise Exception(f"Anime {name} already exists in dictionary")
        anime_episodes[name] = episodes
    except Exception as e:
        print(anime_name,e)
        
    time.sleep(0.4)

In [20]:
AVERAGE_ANIME_EPISODE_MINUTES=20
total_episodes_seen=0
for key,value in anime_episodes.items():
    try:
        total_episodes_seen+=int(value)
    except Exception as e:
        print(e)
        
total_episodes_seen

12

In [21]:
save_data("anime_episodes_seen.json", anime_episodes)
total_hours_wasted_seen=(total_episodes_seen*AVERAGE_ANIME_EPISODE_MINUTES)//60
print("hours wasted",total_hours_wasted_seen)

tv_anime_list=[x.strip().lower() for x in anime_episodes]
tv_anime_list=sorted(tv_anime_list)
print(*tv_anime_list,sep=', ')

hours wasted 4
kino no tabi: the beautiful world - the animated series


# Games
Using how long to beat website for scraping to extract main story duration in hours

In [22]:
all_games_raw=""" all games here """

In [23]:
all_games = all_games_raw.split(',')
all_games = [x.strip().replace('\n',"") for x in all_games]
games_played=[x for x in all_games if len(x)>0]


In [24]:
HLTB_URL = 'https://howlongtobeat.com/search_results?page=1'

headers={
    'content-type': 'application/x-www-form-urlencoded',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'
}

def get_page(title):
    data={
        'queryString':title,
        't':'games',
        'sorthead':'popular',
    }
    return requests.post(HLTB_URL,data=data,headers=headers)

print(get_page('prison'))

def get_game_hours(title):
    try:
        soup=bs(get_page(title).content)
    except Exception as e:
        print(title,'error html call',e)
    try:
        list_div = soup.findAll("div", class_="search_list_details")[0]
        # print(list_div)
        game_title = list_div.find('a').get_text()
        # print(game_title)
        main_story_hours = list_div.findAll("div", class_="search_list_tidbit")[1].get_text()
        number_hours=[x for x in main_story_hours if x.isdigit()]
        number_hours=int("".join(number_hours))
        return (game_title,number_hours)
    except Exception as e:
        print(f'error with finding soup {title}')
        
    # print(list_div)
    # print(number_hours)
    return (None,None)

print(get_game_hours('prison'))


<Response [200]>
('Prison Architect', 15)


In [25]:
game_episodes={}
for index,game_name in enumerate(games_played):
    # if(index == 5):
    #     break
    try:
        name,hours=get_game_hours(game_name)
        if name is None : 
            continue
        if name in game_episodes:
            raise Exception(f"Played game {name} already exists in dictionary")
        game_episodes[name.lower()] = hours
    except Exception as e:
        print(game_name,e)
        
    time.sleep(1)
    
game_episodes
    

error with finding soup all games here


{}

In [26]:
total_hours_wasted=0
for key,value in game_episodes.items():
    try:
        total_hours_wasted+=value
    except Exception as e:
        print(e)
        
save_data("game_hours_wasted_me.json", game_episodes)
total_hours_wasted

0

In [27]:

games_played_list=[x.strip() for x in game_episodes]
games_played_list=sorted(games_played_list)
print(*games_played_list,sep=', ')